In [7]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import count
from pyspark.sql.functions import col
from pyspark.sql.functions import avg

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline



In [8]:
spark = (SparkSession.builder
        .appName("desafio_final")
        .getOrCreate())

In [9]:
data = spark.read.csv("F:/Documentos/0.XP EDUCACAO/Módulo 2 - Bootcamp cientista de Dados/00.Desafio final/stroke_data.csv", sep=",", inferSchema=True, header=True)

In [10]:
#Quantos registros existem no arquivo?

print(data.count())

67135


In [11]:
# Quantas colunas existem no arquivo? Quantas são numéricas? Ao ler o arquivo com spark.read.csv, 
# habilite inferSchema=True. Use a função printSchema() da API de Dataframes.

num_columns = len(data.columns)
num_columns

12

In [12]:
data.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [13]:
# No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?

Stroke = data.groupBy("stroke").agg({"stroke": "count"})
Stroke.show()

+------+-------------+
|stroke|count(stroke)|
+------+-------------+
|     1|        40287|
|     0|        26848|
+------+-------------+



In [14]:
# A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e 
# a seguir use spark.sql para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por 
# tipo de trabalho (work_type). Quantos pacientes sofreram derrame e trabalhavam respectivamente, no setor privado, 
# de forma independente, no governo e quantas são crianças?

data.createOrReplaceTempView('table')

spark.sql("select work_type,count(*) from table where stroke = 1 group by work_type").show()


+-------------+--------+
|    work_type|count(1)|
+-------------+--------+
| Never_worked|      85|
|Self-employed|   10807|
|      Private|   23711|
|     children|     520|
|     Govt_job|    5164|
+-------------+--------+



In [15]:
# Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de 
# participantes do estudo. A maioria dos participantes é:

spark.sql("select gender,count(*) from table group by gender").show()

+------+--------+
|gender|count(1)|
+------+--------+
|Female|   39530|
| Other|      11|
|  Male|   27594|
+------+--------+



In [16]:
# Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: 
# hipertensos ou não-hipertensos. Você pode escrever uma consulta para cada grupo. 
# A partir das probabilidades que você obteve, você conclui que:

# hipertensos

# filtrando hipertensos
data_hyper = data.filter(col("hypertension") == 1)
# contando hipertensos
count_hyper = data.filter(col("hypertension") == 1).count()


hyper = data_hyper.groupBy("stroke").agg(count("*").alias("Count"))

hyper_percent = hyper.withColumn("%", col("Count")/ count_hyper * 100 )

hyper_percent.show()


+------+-----+------------------+
|stroke|Count|                 %|
+------+-----+------------------+
|     1| 8817| 80.03086139602432|
|     0| 2200|19.969138603975676|
+------+-----+------------------+



In [17]:
# não hipertensos

# filtrando não hipertensos
data_nhyper = data.filter(col("hypertension") == 0)
# contando hipertensos
count_nhyper = data.filter(col("hypertension") == 0).count()


hyper = data_nhyper.groupBy("stroke").agg(count("*").alias("Count"))

hyper_percent = hyper.withColumn("%", col("Count")/ count_nhyper * 100 )

hyper_percent.show()

+------+-----+-----------------+
|stroke|Count|                %|
+------+-----+-----------------+
|     1|31470|56.07826365871913|
|     0|24648|43.92173634128087|
+------+-----+-----------------+



In [18]:
# Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade. 
# Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?

spark.sql("select age,count(*) from table where stroke = 1 group by age order by count(*) desc").show()

+----+--------+
| age|count(1)|
+----+--------+
|79.0|    2916|
|78.0|    2279|
|80.0|    1858|
|81.0|    1738|
|82.0|    1427|
|77.0|     994|
|74.0|     987|
|63.0|     942|
|76.0|     892|
|70.0|     881|
|66.0|     848|
|75.0|     809|
|67.0|     801|
|57.0|     775|
|73.0|     759|
|65.0|     716|
|72.0|     709|
|68.0|     688|
|69.0|     677|
|71.0|     667|
+----+--------+
only showing top 20 rows



In [19]:
# Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.

# filtrando pessoas com derrames e após 50 anos
age_50 = data.filter((col("age") > 50) & (col("stroke") == 1))

cont_50 = age_50.count()

print(cont_50)

28938


In [20]:
# Usando spark.sql, determine qual o nível médio de glicose para pessoas que, 
# respectivamente, sofreram e não sofreram derrame.


glicose = data.groupBy("stroke").agg(avg("avg_glucose_level").alias("Count"))

glicose.show()

+------+------------------+
|stroke|             Count|
+------+------------------+
|     1|119.95307046938272|
|     0|103.60273130214506|
+------+------------------+



In [21]:
# Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?

imc = data.groupBy("stroke").agg(avg("bmi"))
imc.show()



+------+------------------+
|stroke|          avg(bmi)|
+------+------------------+
|     1|29.942490629729495|
|     0|27.989678933253657|
+------+------------------+



-------------------------------------------- Estudar para melhor entendimento ------------------------------------------------------

In [22]:
# Crie um modelo de árvore de decisão que prevê a chance de derrame (stroke) a partir das variáveis contínuas/categóricas: 
# idade, BMI, hipertensão, doença do coração, nível médio de glicose. Use o conteúdo da segunda aula interativa para criar 
# e avaliar o modelo.

# Prepare as features usando VectorAssembler
feature_columns = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_assembled = assembler.transform(data)

# Divida o conjunto de dados em treino e teste
(training_data, test_data) = df_assembled.randomSplit([0.8, 0.2], seed=1234)

# Crie o modelo de árvore de decisão
dt = DecisionTreeClassifier(labelCol="stroke", featuresCol="features")
model = dt.fit(training_data)

# Faça previsões no conjunto de teste
predictions = model.transform(test_data)

# Avalie o desempenho do modelo usando BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="stroke", rawPredictionCol="rawPrediction")
area_under_curve = evaluator.evaluate(predictions)

# Exiba a Área sob a Curva (Area Under ROC Curve)
print(f"Área sob a Curva (Area Under ROC Curve): {area_under_curve}")

# Exiba a árvore de decisão
print("Árvore de Decisão:")
print(model.toDebugString)


Área sob a Curva (Area Under ROC Curve): 0.6245344446021796
Árvore de Decisão:
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_be047e84d256, depth=5, numNodes=39, numClasses=2, numFeatures=5
  If (feature 0 <= 55.5)
   If (feature 0 <= 13.5)
    If (feature 0 <= 7.5)
     Predict: 0.0
    Else (feature 0 > 7.5)
     If (feature 1 <= 19.95)
      If (feature 2 <= 0.5)
       Predict: 0.0
      Else (feature 2 > 0.5)
       Predict: 1.0
     Else (feature 1 > 19.95)
      Predict: 0.0
   Else (feature 0 > 13.5)
    If (feature 2 <= 0.5)
     If (feature 0 <= 33.5)
      Predict: 0.0
     Else (feature 0 > 33.5)
      If (feature 1 <= 31.25)
       Predict: 1.0
      Else (feature 1 > 31.25)
       Predict: 0.0
    Else (feature 2 > 0.5)
     If (feature 0 <= 46.5)
      If (feature 1 <= 35.349999999999994)
       Predict: 0.0
      Else (feature 1 > 35.349999999999994)
       Predict: 1.0
     Else (feature 0 > 46.5)
      Predict: 1.0
  Else (feature 0 > 55.5)
   If (feature

In [23]:


# Avalie o desempenho do modelo usando MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="stroke", 
    predictionCol="prediction", 
    metricName="accuracy"
)

# Calcule a acurácia
accuracy = evaluator.evaluate(predictions)

# Exiba a acurácia
print(f"Acurácia do Modelo: {accuracy}")



Acurácia do Modelo: 0.6901492537313433


In [24]:
# Adicione ao modelo as variáveis categóricas: gênero e status de fumante. 
# Use o conteúdo da aula interativa para lidar com as variáveis categóricas.  
# A acurácia (qualidade) do modelo aumentou para:


# Indexação das variáveis categóricas
gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
smoking_status_indexer = StringIndexer(inputCol="smoking_status", outputCol="smoking_status_index")

# Codificação OneHot das variáveis indexadas
gender_encoder = OneHotEncoder(inputCol="gender_index", outputCol="gender_encoded")
smoking_status_encoder = OneHotEncoder(inputCol="smoking_status_index", outputCol="smoking_status_encoded")

# Prepare as features usando VectorAssembler
feature_columns = [
    "age", "bmi", "hypertension", "heart_disease", "avg_glucose_level",
    "gender_encoded", "smoking_status_encoded"
]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Crie o modelo de árvore de decisão
dt = DecisionTreeClassifier(labelCol="stroke", featuresCol="features")

# Construa o pipeline
pipeline = Pipeline(stages=[gender_indexer, smoking_status_indexer, gender_encoder, smoking_status_encoder, assembler, dt])

# Divida o conjunto de dados em treino e teste
(training_data, test_data) = data.randomSplit([0.8, 0.2], seed=1234)

# Treine o modelo
model = pipeline.fit(training_data)

# Faça previsões no conjunto de teste
predictions = model.transform(test_data)

# Avalie o desempenho do modelo usando MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="stroke", 
    predictionCol="prediction", 
    metricName="accuracy"
)

# Calcule a acurácia
accuracy = evaluator.evaluate(predictions)

# Exiba a acurácia
print(f"Acurácia do Modelo com Variáveis Categóricas: {accuracy}")



Acurácia do Modelo com Variáveis Categóricas: 0.8373880597014925


In [25]:

# Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu?

# Treine o modelo
model = pipeline.fit(training_data)

# Acesse as importâncias das variáveis
importances = model.stages[-1].featureImportances.toArray()

# Crie um dicionário associando o nome da variável à sua importância
feature_importance_dict = dict(zip(feature_columns, importances))

# Exiba a importância das variáveis
print("Importância das Variáveis:")
for feature, importance in sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True):
    print(f"{feature}: {importance}")


Importância das Variáveis:
age: 0.16916221419073177
avg_glucose_level: 0.00841995578767354
gender_encoded: 0.0007725059616589108
bmi: 0.0007348283708603001
hypertension: 0.0
heart_disease: 0.0
smoking_status_encoded: 0.0


In [26]:
# Adicione ao modelo as variáveis categóricas: gênero e status de fumante. 
# Use o conteúdo da aula interativa para lidar com as variáveis categóricas. 
# Qual a profundidade da árvore de decisão? 


# Treine o modelo
model = pipeline.fit(training_data)

# Acesse a profundidade da árvore
tree_depth = model.stages[-1].getOrDefault("maxDepth")

# Exiba a profundidade da árvore
print(f"Profundidade da Árvore de Decisão: {tree_depth}")


Profundidade da Árvore de Decisão: 5


In [27]:
# Quantos nodos a árvore de decisão possui?

# Treine o modelo
model = pipeline.fit(training_data)

# Acesse o número de nós na árvore
num_nodes = model.stages[-1].numNodes

# Exiba o número de nós na árvore
print(f"Número de Nós na Árvore de Decisão: {num_nodes}")


Número de Nós na Árvore de Decisão: 25
